# Company Brochure with GenAI

In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
fair_supply = Website("https://www.linkedin.com/company/fairsupply-com-au-pty-limited/")
print(fair_supply.get_contents())

Webpage Title:
Fair Supply | LinkedIn
Webpage Contents:
Skip to main content
LinkedIn
Articles
People
Learning
Jobs
Games
Get the app
Join now
Sign in
Fair Supply
Technology, Information and Internet
Sydney, NSW
4,202 followers
Simplify your ESG Risk Management and Compliance
Follow
Discover all 41 employees
Report this company
About us
Fair Supply is a cloud-based ESG risk management and compliance solution.

We are a purpose driven ESG technology and data business, specialising in identifying supply chain ESG risk. We help companies identify, manage and mitigate their risk and exposure to modern slavery, greenhouse gas (GHG) carbon emissions, across Scope 1, 2 and 3 and biodiversity loss risk.

We simplify ESG risk management and compliance by using business data combined with supply chain mapping technology to reduce time, cost, and effort. 

Our founders are a human rights lawyer and an industrial mathematician. Subject matter expertise and precision are at the core of our technolo

In [5]:
link_system_prompt = (
    "You are provided with a list of links found on a LinkedIn page. "
    "Decide which of these links are most relevant for a thorough company analysis—"
    "for example, a company profile, job postings, or employee profiles. "
    "Exclude Terms of Service, Privacy policy, or email links. "
    "You should respond in JSON as in this example:"
)
link_system_prompt += """
{
    "links": [
        {"type": "company profile", "url": "https://www.linkedin.com/company/example-company"},
        {"type": "job posting", "url": "https://www.linkedin.com/jobs/view/123456/"},
        {"type": "employee profile", "url": "https://www.linkedin.com/in/john-doe"}
    ]
}
"""



In [6]:
print(link_system_prompt)

You are provided with a list of links found on a LinkedIn page. Decide which of these links are most relevant for a thorough company analysis—for example, a company profile, job postings, or employee profiles. Exclude Terms of Service, Privacy policy, or email links. You should respond in JSON as in this example:
{
    "links": [
        {"type": "company profile", "url": "https://www.linkedin.com/company/example-company"},
        {"type": "job posting", "url": "https://www.linkedin.com/jobs/view/123456/"},
        {"type": "employee profile", "url": "https://www.linkedin.com/in/john-doe"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = (
        f"Here is the list of links on the website of {website.url}. "
        "Please decide which of these are relevant for a thorough company analysis. "
        "Specifically, we are looking for pages or resources that help assess the company's background, mission, "
        "values, leadership, culture, finances, or job opportunities. "
        "Provide your response in JSON format, listing the full https URLs. "
        "Do not include Terms of Service, Privacy policy, or email links.\n"
        "Links (some might be relative links):\n"
    )
    user_prompt += "\n".join(website.links)
    return user_prompt


In [8]:
print(get_links_user_prompt(fair_supply))

Here is the list of links on the website of https://www.linkedin.com/company/fairsupply-com-au-pty-limited/. Please decide which of these are relevant for a thorough company analysis. Specifically, we are looking for pages or resources that help assess the company's background, mission, values, leadership, culture, finances, or job opportunities. Provide your response in JSON format, listing the full https URLs. Do not include Terms of Service, Privacy policy, or email links.
Links (some might be relative links):
#main-content
/?trk=organization_guest_nav-header-logo
https://www.linkedin.com/pulse/topics/home/?trk=organization_guest_guest_nav_menu_articles
https://www.linkedin.com/pub/dir/+/+?trk=organization_guest_guest_nav_menu_people
https://www.linkedin.com/learning/search?trk=organization_guest_guest_nav_menu_learning
https://www.linkedin.com/jobs/search?trk=organization_guest_guest_nav_menu_jobs
https://www.linkedin.com/games?trk=organization_guest_guest_nav_menu_games
ms-windows

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
fair_supply.links


['#main-content',
 '/?trk=organization_guest_nav-header-logo',
 'https://www.linkedin.com/pulse/topics/home/?trk=organization_guest_guest_nav_menu_articles',
 'https://www.linkedin.com/pub/dir/+/+?trk=organization_guest_guest_nav_menu_people',
 'https://www.linkedin.com/learning/search?trk=organization_guest_guest_nav_menu_learning',
 'https://www.linkedin.com/jobs/search?trk=organization_guest_guest_nav_menu_jobs',
 'https://www.linkedin.com/games?trk=organization_guest_guest_nav_menu_games',
 'ms-windows-store://pdp/?ProductId=9WZDNCRFJ4Q7&mode=mini&cid=guest_nav_upsell&trk=organization_guest_guest_nav_menu_windows',
 'https://www.linkedin.com/signup?session_redirect=https%3A%2F%2Fwww%2Elinkedin%2Ecom%2Fcompany%2Ffairsupply-com-au-pty-limited&trk=organization_guest_nav-header-join',
 'https://www.linkedin.com/login?session_redirect=https%3A%2F%2Fwww%2Elinkedin%2Ecom%2Fcompany%2Ffairsupply-com-au-pty-limited&fromSignIn=true&trk=organization_guest_nav-header-signin',
 'https://www.link

In [11]:
# Checking links from LinkedIn on Fair Supply LinkedIn page
get_links("https://www.linkedin.com/company/fairsupply-com-au-pty-limited/")

{'links': [{'type': 'company profile',
   'url': 'https://au.linkedin.com/company/fairsupply-com-au-pty-limited'},
  {'type': 'job posting',
   'url': 'https://www.linkedin.com/jobs/search?trk=organization_guest_guest_nav_menu_jobs'},
  {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/cwkwong'},
  {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/jvullinghs'},
  {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/benderson'},
  {'type': 'employee profile',
   'url': 'https://au.linkedin.com/in/denvinlwong'},
  {'type': 'company funding',
   'url': 'https://www.crunchbase.com/organization/fairsupply/funding_rounds/funding_rounds_list?utm_source=linkedin&utm_medium=referral&utm_campaign=linkedin_companies&utm_content=all_fundings_anon&trk=funding_all-rounds'},
  {'type': 'company funding',
   'url': 'https://www.crunchbase.com/funding_round/fairsupply-series-a--760cf817?utm_source=linkedin&utm_medium=referral&utm_campaign=linkedin_companies&utm_co

## Second step: Company analysis

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "LinkedIn:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://www.linkedin.com/company/fairsupply-com-au-pty-limited/"))

Found links: {'links': [{'type': 'company profile', 'url': 'https://au.linkedin.com/company/fairsupply-com-au-pty-limited'}, {'type': 'job posting', 'url': 'https://www.linkedin.com/jobs/search?trk=organization_guest_guest_nav_menu_jobs'}, {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/cwkwong'}, {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/jvullinghs'}, {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/benderson'}, {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/denvinlwong'}, {'type': 'news article', 'url': 'https://www.linkedin.com/posts/fairsupply-com-au-pty-limited_google-for-esg-fair-supply-raises-12m-activity-7285533652698963971-Dt6y'}, {'type': 'news article', 'url': 'https://www.linkedin.com/posts/fairsupply-com-au-pty-limited_spotlight-due-diligence-by-fair-supply-activity-7239437495967350784-o6eN'}, {'type': 'news article', 'url': 'https://www.linkedin.com/posts/fairsupply-com-au-pty-limited_the-australian-gov

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website, \
focusing on a thorough company analysis. Provide a concise overview of the company’s background, mission, \
values, leadership, workplace culture, and job opportunities. Respond in Markdown format. \
Include any details that would be helpful for a prospective candidate evaluating the company, \
such as career growth, employee development, or insights into day-to-day life, if available."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_company_analysis_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += (
        "Here are the contents of its linkedin page and other relevant pages. "
        "Use this information to create a concise overview of the company that focuses on a thorough company analysis. "
        "Highlight the company's mission, values, leadership, culture, and job opportunities—"
        "any insights relevant for someone evaluating this company as a potential employer. "
        "Respond in Markdown.\n"
    )
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000]  # Truncate if more than 20,000 characters
    return user_prompt


In [16]:
get_company_analysis_user_prompt("Fair Supply", "https://www.linkedin.com/company/fairsupply-com-au-pty-limited/")

Found links: {'links': [{'type': 'company profile', 'url': 'https://au.linkedin.com/company/fairsupply-com-au-pty-limited'}, {'type': 'job posting', 'url': 'https://www.linkedin.com/jobs/search?trk=organization_guest_guest_nav_menu_jobs'}, {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/cwkwong'}, {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/jvullinghs'}, {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/benderson'}, {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/denvinlwong'}, {'type': 'company background', 'url': 'https://www.crunchbase.com/organization/fairsupply?utm_source=linkedin&utm_medium=referral&utm_campaign=linkedin_companies&utm_content=profile_cta_anon&trk=funding_crunchbase'}, {'type': 'company funding', 'url': 'https://www.crunchbase.com/organization/fairsupply/funding_rounds/funding_rounds_list?utm_source=linkedin&utm_medium=referral&utm_campaign=linkedin_companies&utm_content=all_fundings_anon&trk=fundin

'You are looking at a company called: Fair Supply\nHere are the contents of its linkedin page and other relevant pages. Use this information to create a concise overview of the company that focuses on a thorough company analysis. Highlight the company\'s mission, values, leadership, culture, and job opportunities—any insights relevant for someone evaluating this company as a potential employer. Respond in Markdown.\nLinkedIn:\nWebpage Title:\nFair Supply | LinkedIn\nWebpage Contents:\nSkip to main content\nLinkedIn\nArticles\nPeople\nLearning\nJobs\nGames\nGet the app\nJoin now\nSign in\nFair Supply\nTechnology, Information and Internet\nSydney, NSW\n4,202 followers\nSimplify your ESG Risk Management and Compliance\nFollow\nView all 41 employees\nReport this company\nAbout us\nFair Supply is a cloud-based ESG risk management and compliance solution.\n\nWe are a purpose driven ESG technology and data business, specialising in identifying supply chain ESG risk. We help companies identify

In [17]:
def create_company_analysis(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_company_analysis_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [18]:
create_company_analysis("Fair Supply", "https://www.linkedin.com/company/fairsupply-com-au-pty-limited/")

Found links: {'links': [{'type': 'company profile', 'url': 'https://au.linkedin.com/company/fairsupply-com-au-pty-limited'}, {'type': 'job posting', 'url': 'https://www.linkedin.com/jobs/search?trk=organization_guest_guest_nav_menu_jobs'}, {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/cwkwong'}, {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/jvullinghs'}, {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/benderson'}, {'type': 'employee profile', 'url': 'https://au.linkedin.com/in/denvinlwong'}, {'type': 'company news', 'url': 'https://www.linkedin.com/posts/fairsupply-com-au-pty-limited_spotlight-due-diligence-by-fair-supply-activity-7239437495967350784-o6eN'}, {'type': 'company news', 'url': 'https://www.linkedin.com/posts/fairsupply-com-au-pty-limited_google-for-esg-fair-supply-raises-12m-activity-7285533652698963971-Dt6y'}, {'type': 'company news', 'url': 'https://www.linkedin.com/posts/fairsupply-com-au-pty-limited_australias-mandato

# Fair Supply Company Overview

## Background
**Fair Supply** is a Sydney-based, privately held company founded in **2019**. With a focus on technology and sustainability, it offers a **cloud-based ESG (Environmental, Social, and Governance) risk management and compliance solution**. The company specializes in identifying and addressing supply chain ESG risks, such as modern slavery, greenhouse gas emissions (across Scope 1, 2, and 3), and biodiversity loss.

## Mission
Fair Supply aims to **simplify ESG risk management and compliance** for companies, enabling them to effectively manage their exposure to environmental and social risks. Their solutions blend business data with innovative supply chain mapping technology to streamline this process.

## Values
1. **Purpose-driven Focus**: The company is committed to making a positive impact in the field of ESG compliance and risk management.
2. **Expertise and Precision**: Founded by a human rights lawyer and an industrial mathematician, Fair Supply relies on subject matter expertise to guide its technological solutions.

## Leadership
Fair Supply's leadership team includes its co-founders:
- **Kimberly Randle (CEO)**: A human rights lawyer focused on ethical compliance and supply chain transparency.
- **Dr. Arne Geschke (CTO)**: An expert in industrial mathematics bringing data-driven insights to the ESG landscape.

## Workplace Culture
Fair Supply promotes a culture of **innovation**, **collaboration**, and **social responsibility**. Employees are encouraged to engage with projects that align with the company’s mission to combat modern slavery and reduce environmental impacts. The company’s focus on education is evident through offerings like webinars and training sessions to enhance team capabilities in ESG practices.

## Job Opportunities
Fair Supply, with a current team of **11-50 employees**, is in a growth phase, expanding into new markets like North America and Europe. Open positions typically range from technical roles (e.g., software engineers) to strategic positions in risk intelligence and sustainability. 

### Career Growth and Employee Development
- **Training and Education**: The company is proactive in facilitating professional development through workshops and training programs tailored to ESG skills.
- **Growth Potential**: As Fair Supply expands its operations internationally, there may be ample opportunities for employees to advance their careers within an exciting growth environment.

### Day-to-Day Insights
Employees at Fair Supply are likely engaged in developing innovative solutions for complex ESG challenges, collaborating with partners, and working on cutting-edge technologies. The environment is dynamic, focused on tackling real-world issues through technology.

## Conclusion
Fair Supply presents a compelling choice for candidates interested in sustainability, technology, and social impact. With a strong commitment to ethical practices and innovative solutions, it offers a meaningful work environment where individuals can contribute to significant advancements in ESG compliance and risk management.

## LinkedIn connections

In [19]:
link_system_prompt_people = (
    "You are provided with a list of links found on a LinkedIn page. "
    "Decide which of these links are employee profiles that might be relevant for someone wanting to connect "
    "and learn more about their experience. "
    "Exclude Terms of Service, Privacy policy, or email links. "
    "You should respond in JSON as in this example:"
)
link_system_prompt_people += """
{
    "links": [
        {"type": "employee profile", "url": "https://www.linkedin.com/in/john-doe"}
    ]
}
"""


In [20]:
def get_people_links_user_prompt(website):
    user_prompt = (
        f"Here is the list of links on the website of {website.url}. "
        "Please decide which of these links appear to be personal or employee profiles relevant for learning about people who work there. "
        "Provide your response in JSON format, listing the full https URLs. "
        "Do not include Terms of Service, Privacy policy, or email links.\n"
        "Links (some might be relative links):\n"
    )
    user_prompt += "\n".join(website.links)
    return user_prompt


In [21]:
def get_people_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt_people},
            {"role": "user", "content": get_people_links_user_prompt(website)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)


In [22]:
def get_all_people_details(url):
    """
    Retrieves the LinkedIn page, extracts personal/employee profile links,
    and then optionally fetches more detail from each profile page (if it’s publicly visible).
    """
    result = "LinkedIn People:\n"
    
    # Step 1: Scrape the main page for personal profile links
    people_links_result = get_people_links(url)
    
    # Step 2: For each link, fetch the profile contents (caution: possible TOS violation)
    for link in people_links_result["links"]:
        if link["type"] == "employee profile":
            # Potentially scrape the individual's profile page
            person_url = link["url"]
            person_site = Website(person_url)  # Might or might not succeed depending on if public
            result += f"\n\nEmployee Profile: {person_url}\n"
            result += person_site.get_contents()
    
    return result


In [30]:
def get_company_people_analysis(company_name, url):
    """
    Pulls employee pages from the main LinkedIn URL, 
    then feeds those details to the model for a summary of employee roles/areas of expertise.
    Stream the response and display it progressively in Markdown.
    """
    # 1. Gather employee details (scraped content).
    people_details = get_all_people_details(url)

    # 2. Build the user prompt.
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += (
        "Here is information about people from their LinkedIn page. "
        "Based on this information, provide a short overview of the roles/titles and areas of expertise "
        "these employees appear to have. Respond in Markdown.\n"
    )
    user_prompt += people_details

    # 3. Truncate if too large.
    user_prompt = user_prompt[:20_000]

    # 4. Start a streamed chat completion request.
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You are an assistant that provides a concise summary of employee information."},
            {"role": "user", "content": user_prompt}
        ],
        stream=True
    )
    
    # 5. Build and display the response incrementally.
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    
    for chunk in stream:
        chunk_text = chunk.choices[0].delta.content or ""
        response += chunk_text
        # Remove code fences or "markdown" hints if desired
        display_text = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(display_text), display_id=display_handle.display_id)
    
    

In [32]:
get_company_people_analysis("Fair Supply", "https://www.linkedin.com/company/fairsupply-com-au-pty-limited/people/")

It seems like there was an error, as no specific LinkedIn information about employees from Fair Supply was provided. However, I can help you structure a summary layout for the information once you provide it. Here's an example template you can use:


# Fair Supply Employee Overview

## Roles/Titles
- **[Title/Role 1]**: [Brief description of the role]
- **[Title/Role 2]**: [Brief description of the role]
- **[Title/Role 3]**: [Brief description of the role]
  
## Areas of Expertise
- **[Expertise Area 1]**: [Details about expertise]
- **[Expertise Area 2]**: [Details about expertise]
- **[Expertise Area 3]**: [Details about expertise]


Once you provide the specific roles and areas of expertise, I can fill in the details accordingly!